In [14]:
import re
class SimpleTokenizerV1:
    def __init__(self,vocab):
        self.str_to_int = vocab
        ##This is just a reverse dictionary
        self.int_to_str = { i : s for s, i in vocab.items()}
    
    def encode(self, text):
        """Splitting the text into tokens and encoding them"""
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]

        ##Adding the <UNK> tags check in vocab.
        ## The tags have to be present in the vocab, this is just a placeholder.
        preprocessed = [item if item in self.str_to_int else "<|unk|>" for item in preprocessed]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids]) 
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [30]:
with open("Training material/the-verdict.txt", "r", encoding= "utf-8") as file:
    raw_text = file.read()

"""The tokenised library for the book has been generated using a simple regex"""
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]

token_set = sorted(set(preprocessed))
vocab_size = len(token_set)
print(vocab_size)

vocab = {token:integer for integer,token in enumerate(token_set)}
for i, item in enumerate(vocab.items()):
    if i >= 50:
        break


1130


In [16]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token:integer for integer,token in enumerate(all_tokens)}

print(len(vocab.items()))

##Testing the tokenizer
tokenizer = SimpleTokenizerV1(vocab)
text = """"It's the last he painted, you know," 
       Mrs. Gisburn said with pardonable pride. maitai"""
ids = tokenizer.encode(text)


1132


In [17]:
##Testing tiktoken.

from importlib.metadata import version
import tiktoken

##Get the encoding set for GPT2 (Byte pair encoding algorithm)
tokenizer = tiktoken.get_encoding("gpt2")

## Tokenizing entire verdict.txt
enc_text = tokenizer.encode(raw_text)

##Test data
enc_sample = enc_text[50:]
context_size = 4
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

In [19]:
"""Dataset and Dataloader classes"""
import torch
from torch.utils.data import Dataset, DataLoader
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt)    

        for i in range(0, len(token_ids) - max_length, stride):     
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):         
        return self.input_ids[idx], self.target_ids[idx]
    
    def create_dataloader_v1(txt, batch_size=4, max_length=256,
                            stride=128, shuffle=True, drop_last=True,
                            num_workers=0):
        tokenizer = tiktoken.get_encoding("gpt2")                         
        dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)   
        dataloader = DataLoader(
            dataset,
            batch_size=batch_size,
            shuffle=shuffle,
            drop_last=drop_last,     
            num_workers=num_workers     
        )
        return dataloader

In [29]:
with open("Training material/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

dataLoader = create_dataloader_v1(raw_text,batch_size=2,max_length=8,stride=4, shuffle=False)
data_iter = iter(dataLoader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[   40,   367,  2885,  1464,  1807,  3619,   402,   271],
        [ 1807,  3619,   402,   271, 10899,  2138,   257,  7026]]), tensor([[  367,  2885,  1464,  1807,  3619,   402,   271, 10899],
        [ 3619,   402,   271, 10899,  2138,   257,  7026, 15632]])]


In [35]:
#Experimenting positional embeddings.
vocabulary_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocabulary_size, output_dim)

max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
   stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

token_embeddings = token_embedding_layer(inputs)
print(token_embeddings)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])
tensor([[[-1.0197, -2.3440, -1.2713,  ..., -0.3323,  0.0762,  0.3855],
         [ 0.4505,  0.8796, -0.6015,  ..., -0.0715, -0.3378, -0.3569],
         [-1.8940, -0.6210,  1.5341,  ..., -0.3633, -0.6550,  0.0658],
         [-1.1656, -0.6339, -1.3029,  ..., -1.5551, -1.5283,  1.3511]],

        [[ 0.9928,  0.0772, -0.8668,  ...,  2.4724,  0.0343,  1.0992],
         [-0.0391, -2.6853,  0.8640,  ..., -0.3042, -0.1135, -1.2823],
         [ 0.1447, -0.5722, -0.0729,  ...,  0.4139,  0.2392, -1.0613],
         [-0.3878,  0.1036, -0.6307,  ..., -1.1176,  0.7952,  0.4843]],

        [[-0.6663, -1.1485, -2.0999,  ...,  0.1627,  0.7526,  0.2589],
     